# 1. Problem(3~4line)
1. Definition of Input, Output
2. classificaiton,
3. How many number of labels?
4. How the data looks like?

=============================================\
Input data는 column 이고, Output은 target ~이다. 해당 문제는 분류(classification)이다. 정답(label)의 경우 51개 이며, 27개와 24개의 비율을 보인다. 데이터는 전반적으로 continual feature보다 object의 categorical feature가 많은 것을 확인할 수 있음. 이를 적절한 Label encoding or one-hot encoding을 거치고 모델을 학습해야할 것이다.

# 2. Feature(3~4line)
1. define feature by your self
2. How did you preprocess the data

=============================================\
feature 중 필요 없는 ~ column들을 제거하였다. Date의 경우 특정 시점에서 뺴주었다.

~의 feature의 경우 missing value같은 경우 최빈값으로 대체하였다.

원핫인코딩 시 row보다 feature의 개수가 많아 데이터의 설명력이 떨어지고, sparse data해지므로, 이는 차원의 저주를 야기할 수 있다. 따라서 PCA를 거쳐 feature dimension을 reduction하였다. 



# 3. Model()
1. Why did you choose your models?
2. Model list : Logistic regression, Support Vector Machine, Random Forest, Decision Tree, Multilayer perceptron

=============================================\
분류 모델 이므로, classification의 모델인 ~ 을 사용하였다. 

# 4. Measure
1. Explain the steps of measurement
2. 10-fold cross validation (Classification: "weighted F1 score")

=============================================\
평가척도는 F1 score으로, cross_val을 통해 10-fold를 적용하였다.

# 5. Model parameter engineering
1.  How did you change parameters to improve performance?
Explain the reason
2. Check the data imbalance, data sparsity, ...
				- Explain how you handel this
				- Note that high performance is not always good; for example, you may get 90~95% because of severe data imbalance. How can we handle this problem?
                
=============================================\
grid searchCV를 통해 최적의 파라미터를 찾아주었다.
타겟값의 비율은 27:24로, imbalance하지 않아 sampling 기법을 적용하지 않았다. 



In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier

In [68]:
train = pd.read_csv('./data/Pakistan.csv', encoding = 'unicode_escape')
print(train.shape)
train

(51, 18)


,S#,Politician,Day,Date,Day Type,Time,City,Location,Location Category,Province,Latitude,Longititude,Target Category,Space (Open/Closed),Party,Target Status,Killed,Injured
0,1,Liaqat Ali Khan,Tuesday,16-Oct-51,Working Day,Evening,Rawalpindi,"Company Bagh (Company Gardens), Rawalpindi",Park/Ground,Punjab,33.605800,73.043700,Target,Open,Alll India Muslim League,Killed,1,1
1,2,Khan abdul Jabbar Khan,Friday,9-May-58,Working Day,Morning,Lahore,"Sadullah Khan's house at 16 Aikman Road, GOR",Residence,Punjab,31.482635,74.071272,Target,Open,Chief Minister of West Pakistan,Killed,1,0
2,3,Hayyat Sherpao,Friday,8-Feb-85,Working Day,Evening,Peshawar,Campus of University of Peshawar,University,KPK,33.999533,71.425020,Target,Open,PPPP,Killed,1,0
3,4,Najeeb Ahmed,Friday,6-Apr-90,Working Day,After Noon,Karachi,North Nazimabad,Residence,Sindh,24.991800,66.991100,Target,Closed,Student wing of PPP,Killed,1,0
4,5,Azeem Ahmed Tariq,Saturday,1-May-93,Working Day,Morning,Karachi,near his house,Residence,Sindh,24.991800,66.991100,Target,Closed,MQM,Killed,1,0
5,6,Hakim Muhammad Said,Saturday,17-Oct-98,Working Day,After Noon,Karachi,Details Missing,Details Missing,Sindh,24.991800,66.991100,Target,Open,Independent,Killed,1,0
6,7,Maulana Azam Tariq,Monday,6-Oct-03,Working Day,Evening,Islamabad,"As his car drove into the capital, Islamabad",Road,Capital,33.718000,73.071800,Target,Open,Millat-i-Islamia,Killed,4,0
7,8,Pervez Musharaf,Monday,15-Dec-03,Working Day,Evening,Rawalpindi,A powerful bomb went off minutes after his hig...,Road,Punjab,33.605800,73.043700,Target,Open,All Pakistan Muslim League,Escaped,0,0
8,9,Pervez Musharaf,Thursday,25-Dec-03,Working Day,Evening,Rawalpindi,Attack on Musharaf's Rally,Road,Punjab,33.605800,73.043700,Suicide,Open,All Pakistan Muslim League,Escaped,16,4
9,10,Shaukat Aziz,Friday,30-Jul-04,Working Day,Night,Attock,Jaffar village in fateh jang,Park/Ground,Punjab,33.764500,72.366700,Suicide,Open,PMLQ,Escaped,7,70


In [31]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51 entries, 0 to 50
Data columns (total 18 columns):
S#                     51 non-null int64
Politician             51 non-null object
Day                    51 non-null object
Date                   51 non-null object
Day Type               51 non-null object
Time                   51 non-null object
City                   51 non-null object
Location               51 non-null object
Location Category      48 non-null object
Province               51 non-null object
Latitude               51 non-null float64
Longititude            51 non-null float64
Target Category        51 non-null object
Space (Open/Closed)    51 non-null object
Party                  51 non-null object
Target Status          51 non-null object
Killed                 51 non-null int64
Injured                51 non-null int64
dtypes: float64(2), int64(3), object(13)
memory usage: 7.3+ KB


In [35]:
train['Target Category'].value_counts()

Suicide    27
Target     24
Name: Target Category, dtype: int64

In [56]:
train.drop(['S#', 'Politician'], axis = 1, inplace = True)

In [57]:
object_column = train.dtypes[train.dtypes == 'object'].index.tolist()
for column in object_column:
    print("=================== " + column + " ===================")
    print(train[column].unique())

=================== Day ===================
['Tuesday' 'Friday' 'Saturday' 'Monday' 'Thursday' 'Wednesday' 'Sunday']
=================== Date ===================
['16-Oct-51' '9-May-58' '8-Feb-85' '6-Apr-90' '1-May-93' '17-Oct-98'
 '6-Oct-03' '15-Dec-03' '25-Dec-03' '30-Jul-04' '20-Feb-07' '28-Apr-07'
 '12-Jul-07' '27-Jul-07' '18-Oct-07' '9-Nov-07' '27-Dec-07' '28-Dec-07'
 '9-Feb-08' '11-Feb-08' '16-Feb-08' '6-Oct-08' '26-Jan-09' '17-Dec-09'
 '5-Apr-10' '12-Jul-10' '14-Jul-10' '9-Sep-10' '13-Oct-10' '6-Dec-10'
 '7-Dec-10' '4-Jan-11' '2-Mar-11' '23-Dec-12' '17-Jan-13' '18-Feb-13'
 '16-Apr-13' '1-May-13' '7-May-13' '18-May-13' '3-Jun-13' '18-Jun-13'
 '16-Oct-13' '16-Aug-15' '19-Oct-15' '22-Apr-16' '15-Feb-17' '7-Jul-18'
 '10-Jul-18' '13-Jul-18']
=================== Day Type ===================
['Working Day' 'Holiday']
=================== Time ===================
['Evening' 'Morning' 'After Noon' 'Night' 'Mid Night' 'Noon']
=================== City ===================
['Rawalpindi' 'Laho

In [58]:
object_column = train.dtypes[train.dtypes == 'object'].index.tolist()
for column in object_column:
    print("=================== " + column + " ===================")
    print(train[column].value_counts())

=================== Day ===================
Monday       10
Friday       10
Tuesday       8
Thursday      7
Saturday      7
Wednesday     7
Sunday        2
Name: Day, dtype: int64
=================== Date ===================
13-Jul-18    2
9-Sep-10     1
23-Dec-12    1
7-May-13     1
18-May-13    1
14-Jul-10    1
20-Feb-07    1
16-Apr-13    1
11-Feb-08    1
18-Oct-07    1
16-Feb-08    1
6-Dec-10     1
9-Nov-07     1
5-Apr-10     1
1-May-93     1
10-Jul-18    1
12-Jul-10    1
12-Jul-07    1
16-Oct-51    1
19-Oct-15    1
13-Oct-10    1
28-Dec-07    1
1-May-13     1
16-Aug-15    1
25-Dec-03    1
27-Dec-07    1
9-Feb-08     1
15-Feb-17    1
18-Jun-13    1
26-Jan-09    1
3-Jun-13     1
30-Jul-04    1
6-Oct-03     1
17-Jan-13    1
2-Mar-11     1
16-Oct-13    1
4-Jan-11     1
22-Apr-16    1
6-Oct-08     1
8-Feb-85     1
28-Apr-07    1
7-Jul-18     1
18-Feb-13    1
6-Apr-90     1
9-May-58     1
17-Dec-09    1
15-Dec-03    1
27-Jul-07    1
7-Dec-10     1
17-Oct-98    1
Name: Date, dtype: int64


1. Day 요일 별
2. Date -> 특정 기간에서 빼기
3. Day Type -> Label encoding
4. Time = Label encoding
5. City = One-hot
6. Location  제거
7. Location Category 대신 넣기
8. Province 
9. Space (Open/Closed) : label
10. Party = label
11. Target Status = label


#### Target Category = target 값

# 결측값

In [65]:
train.isnull().sum()
train.fillna('Residence')

,S#,Politician,Day,Date,Day Type,Time,City,Location,Location Category,Province,Latitude,Longititude,Target Category,Space (Open/Closed),Party,Target Status,Killed,Injured
0,1,Liaqat Ali Khan,Tuesday,16-Oct-51,Working Day,Evening,Rawalpindi,"Company Bagh (Company Gardens), Rawalpindi",Park/Ground,Punjab,33.605800,73.043700,Target,Open,Alll India Muslim League,Killed,1,1
1,2,Khan abdul Jabbar Khan,Friday,9-May-58,Working Day,Morning,Lahore,"Sadullah Khan's house at 16 Aikman Road, GOR",Residence,Punjab,31.482635,74.071272,Target,Open,Chief Minister of West Pakistan,Killed,1,0
2,3,Hayyat Sherpao,Friday,8-Feb-85,Working Day,Evening,Peshawar,Campus of University of Peshawar,University,KPK,33.999533,71.425020,Target,Open,PPPP,Killed,1,0
3,4,Najeeb Ahmed,Friday,6-Apr-90,Working Day,After Noon,Karachi,North Nazimabad,Residence,Sindh,24.991800,66.991100,Target,Closed,Student wing of PPP,Killed,1,0
4,5,Azeem Ahmed Tariq,Saturday,1-May-93,Working Day,Morning,Karachi,near his house,Residence,Sindh,24.991800,66.991100,Target,Closed,MQM,Killed,1,0
5,6,Hakim Muhammad Said,Saturday,17-Oct-98,Working Day,After Noon,Karachi,Details Missing,Details Missing,Sindh,24.991800,66.991100,Target,Open,Independent,Killed,1,0
6,7,Maulana Azam Tariq,Monday,6-Oct-03,Working Day,Evening,Islamabad,"As his car drove into the capital, Islamabad",Road,Capital,33.718000,73.071800,Target,Open,Millat-i-Islamia,Killed,4,0
7,8,Pervez Musharaf,Monday,15-Dec-03,Working Day,Evening,Rawalpindi,A powerful bomb went off minutes after his hig...,Road,Punjab,33.605800,73.043700,Target,Open,All Pakistan Muslim League,Escaped,0,0
8,9,Pervez Musharaf,Thursday,25-Dec-03,Working Day,Evening,Rawalpindi,Attack on Musharaf's Rally,Road,Punjab,33.605800,73.043700,Suicide,Open,All Pakistan Muslim League,Escaped,16,4
9,10,Shaukat Aziz,Friday,30-Jul-04,Working Day,Night,Attock,Jaffar village in fateh jang,Park/Ground,Punjab,33.764500,72.366700,Suicide,Open,PMLQ,Escaped,7,70


# 인코딩

In [69]:
# 레이블 인코딩
train.drop(['S#', 'Politician','Date','Location'], axis = 1, inplace = True)

object_columns = train.dtypes[train.dtypes == 'object'].index.tolist()

for column in object_columns:
    train[column] = pd.factorize(train[column])[0]

In [70]:
train.head()

,Day,Day Type,Time,City,Location Category,Province,Latitude,Longititude,Target Category,Space (Open/Closed),Party,Target Status,Killed,Injured
0,0,0,0,0,0,0,33.605800,73.043700,0,0,0,0,1,1
1,1,0,1,1,1,0,31.482635,74.071272,0,0,1,0,1,0
2,1,0,0,2,2,1,33.999533,71.425020,0,0,2,0,1,0
3,1,0,2,3,1,2,24.991800,66.991100,0,1,3,0,1,0
4,2,0,1,3,1,2,24.991800,66.991100,0,1,4,0,1,0


In [71]:
train.drop(['Latitude', 'Longititude'], axis = 1, inplace = True)

# 모델링

In [73]:
Train = train.drop('Target Category', axis = 1, inplace = False)
target = train['Target Category']

    
print(Train.shape , target.shape)

(51, 11) (51,)


In [78]:
rf_clf = RandomForestClassifier(n_estimators=100, random_state = 10)
scores = cross_val_score(rf_clf, Train , target , scoring = 'f1', cv=10)

for iter_count,accuracy in enumerate(scores):
    print("fold {0} F1 score: {1:.4f}".format(iter_count+1, accuracy))

print("Total (Average) F1 score: {0:.4f}".format(np.mean(scores))) 


교차 검증 1 F1 score: 1.0000
교차 검증 2 F1 score: 0.8000
교차 검증 3 F1 score: 0.8571
교차 검증 4 F1 score: 0.8000
교차 검증 5 F1 score: 0.8000
교차 검증 6 F1 score: 1.0000
교차 검증 7 F1 score: 1.0000
교차 검증 8 F1 score: 1.0000
교차 검증 9 F1 score: 0.8000
교차 검증 10 F1 score: 0.6667
평균 F1 score: 0.8724
